# Build station features and exploratory analysis

This notebook processes minute-level GTFS-realtime snapshots, 
builds station-level features and explores the resulting dataset.

In [1]:
from pathlib import Path
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pytz

from metro_disruptions_intelligence.processed_reader import load_rt_dataset
from metro_disruptions_intelligence.features import SnapshotFeatureBuilder

In [2]:
project_root = Path.cwd()
if not (project_root / "notebooks").exists():
    project_root = project_root.parent

processed_rt = project_root / "data" / "processed_final" / "rt"
features_root = project_root / "data" / "stations_features_time_series"
features_root.mkdir(parents=True, exist_ok=True)

In [3]:
processed_rt, features_root

## Available routes

In [4]:
trip_ds = load_rt_dataset(processed_rt, feeds=["trip_updates"])
unique_routes = trip_ds["route_id"].unique()
print("Routes found:", unique_routes)

## Helper functions

In [5]:
def discover_all_snapshot_minutes(root: Path) -> list[int]:
    files = sorted((root / "trip_updates").rglob("trip_updates_*.parquet"))
    minutes = []
    for f in files:
        parts = f.stem.split("_")
        if len(parts) >= 6:
            _, Y, D, M, H, m = parts
            dt = datetime(int(Y), int(M), int(D), int(H), int(m), tzinfo=pytz.UTC)
            minutes.append(int(dt.timestamp()))
    return sorted(minutes)


def compose_path(ts: int, root: Path, feed: str) -> Path:
    dt = datetime.fromtimestamp(ts, tz=pytz.UTC)
    fname = f"{feed}_{dt:%Y-%d-%m-%H-%M}.parquet"
    return (
        root / feed / f"year={dt.year:04d}" / f"month={dt.month:02d}" / f"day={dt.day:02d}" / fname
    )


def build_route_map(root: Path) -> dict:
    first = sorted((root / "trip_updates").rglob("*.parquet"))[0]
    df = pd.read_parquet(first)
    tu = df[["route_id", "direction_id", "stop_id", "stop_sequence"]].drop_duplicates()
    tu = tu.sort_values(["route_id", "direction_id", "stop_sequence"])
    return tu.groupby(["route_id", "direction_id"])["stop_id"].apply(list).to_dict()

## Generate features

In [6]:
minutes = discover_all_snapshot_minutes(processed_rt)
route_map = build_route_map(processed_rt)
builder = SnapshotFeatureBuilder(route_map)

for ts in minutes:
    tu_file = compose_path(ts, processed_rt, "trip_updates")
    vp_file = compose_path(ts, processed_rt, "vehicle_positions")
    if not tu_file.exists() or not vp_file.exists():
        continue
    trip_now = pd.read_parquet(tu_file)
    veh_now = pd.read_parquet(vp_file)
    feats = builder.build_snapshot_features(trip_now, veh_now, ts)
    if feats.empty:
        continue
    feats = feats.reset_index()
    feats["snapshot_timestamp"] = ts
    dt = datetime.fromtimestamp(ts, tz=pytz.UTC)
    out_dir = features_root / f"year={dt.year:04d}" / f"month={dt.month:02d}" / f"day={dt.day:02d}"
    out_dir.mkdir(parents=True, exist_ok=True)
    out_file = out_dir / f"stations_feats_{dt:%Y-%d-%m-%H-%M}.parquet"
    feats.to_parquet(out_file, compression="snappy", index=False)

## Load generated features

In [7]:
feature_files = sorted(features_root.rglob("stations_feats_*.parquet"))
feats_all = pd.concat([pd.read_parquet(f) for f in feature_files], ignore_index=True)
feats_all["snapshot_timestamp"] = pd.to_datetime(
    feats_all["snapshot_timestamp"], unit="s", utc=True
).dt.tz_convert("Australia/Sydney")
feats_all.info()

In [8]:
feats_all.isna().mean()

In [9]:
feats_all.describe()

## Plot headways and delays

In [10]:
days = np.random.choice(feats_all["snapshot_timestamp"].dt.date.unique(), 2, replace=False)
stations = np.random.choice(feats_all["stop_id"].unique(), 2, replace=False)
for stop in stations:
    plt.figure()
    sub = feats_all[feats_all["stop_id"] == stop]
    for day in days:
        day_sub = sub[sub["snapshot_timestamp"].dt.date == day]
        plt.plot(day_sub["snapshot_timestamp"], day_sub["headway_t"], label=str(day))
    plt.title(f"Headway at stop {stop}")
    plt.legend()
    plt.xticks(rotation=45)
    plt.show()
for stop in stations:
    plt.figure()
    sub = feats_all[feats_all["stop_id"] == stop]
    for day in days:
        day_sub = sub[sub["snapshot_timestamp"].dt.date == day]
        plt.plot(day_sub["snapshot_timestamp"], day_sub["arrival_delay_t"], label=str(day))
    plt.title(f"Delay at stop {stop}")
    plt.legend()
    plt.xticks(rotation=45)
    plt.show()